In [78]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv
from selenium import webdriver
import time
import os

In [ ]:
# requests
# for i in range(4):
#   url = f"https://search.daum.net/search?w=tot&q=202{i}%EB%85%84%EC%98%81%ED%99%94%EC%88%9C%EC%9C%84&DA=MOR&rtmaxcoll=MOR"
#   headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"}
#   res = requests.get(url,headers=headers)
#   res.raise_for_status()

#   # Soup변환
#   soup = BeautifulSoup(res.text,"lxml")
#   with open(f"screen{i}.html","w",encoding="UTF-8") as f:
#     f.write(soup.prettify())


In [80]:
# html 넣을 디렉토리 확인
if not os.path.exists("screens"):
  os.mkdir("screens")

In [81]:
# selenium
browser = webdriver.Chrome()
browser.maximize_window()
for i in range(4):
  url = f"https://search.daum.net/search?w=tot&q=202{i}%EB%85%84%EC%98%81%ED%99%94%EC%88%9C%EC%9C%84&DA=MOR&rtmaxcoll=MOR"
  browser.get(url)

  soup = BeautifulSoup(browser.page_source,"lxml")
  with open(f'screens/screen{i}.html','w',encoding='utf-8') as f:
    f.write(soup.prettify())
  time.sleep(2)
browser.close()

In [83]:
# functions
def f_trans(num):
  tail = ''
  if num.find('만명') != -1: tail = '0000'
  return int(''.join([c for c in num if c.isdigit()]) + tail)

def f_subject(title):
  invalid = r'\/:*?"<>| '

  return ''.join([c if c not in invalid else '_' for c in title])


In [85]:
# 이미지 디렉토리 확인
if not os.path.exists("images"):
  os.mkdir("images")

In [86]:
# 데이터 크롤링
mdata = []
for i in range(4):
  with open(f"screens/screen{i}.html", "r", encoding="UTF-8") as f:
    soup = BeautifulSoup(f, "lxml")

    # 1-10위까지 제목, d예매율 출력하시오.
    data = soup.find("c-flicking",{"id":"mor_history_id_0"})
    uls = data.find_all("ul", {"class":"ty_flow35"})

    for ul in uls:
      lis = ul.find_all("li")

      for li in lis:
        m = []

        m.append(int(li.find("span", {"class": "badge-basic"}).text.strip()))
        m.append(li.find("c-title").text.strip())
        m.append(f_trans(li.find("c-contents-desc").text.strip()))
        m.append(li.find("c-thumb")['data-original-src'])
        m.append(li.find("c-contents-desc-sub").text.strip()[:-1])

        with open(f"images/{f_subject(m[1])}.jpg", "wb") as f:
          res = requests.get(m[-2])
          f.write(res.content)

        mdata.append(m)
mdata

[[1,
  '남산의 부장들',
  4750000,
  'https://search1.kakaocdn.net/thumb/R232x328.q85/?fname=http%3A%2F%2Ft1.daumcdn.net%2Fmovie%2F2b07e0c665de4482b9a0094f66bc11761578025532290',
  '2020.01.22'],
 [2,
  '다만 악에서 구하소서',
  4350000,
  'https://search1.kakaocdn.net/thumb/R232x328.q85/?fname=https%3A%2F%2Ft1.daumcdn.net%2Fmovie%2F7a840d47e20d6a00f57194be307ea942e120bf03',
  '2020.10.28'],
 [3,
  '반도',
  3810000,
  'https://search1.kakaocdn.net/thumb/R232x328.q85/?fname=https%3A%2F%2Ft1.daumcdn.net%2Fmovie%2F389d6e403d8ee48c419c7b16908919103e2c2670',
  '2020.07.15'],
 [4,
  '히트맨',
  2400000,
  'https://search1.kakaocdn.net/thumb/R232x328.q85/?fname=http%3A%2F%2Ft1.daumcdn.net%2Fmovie%2F23837d99576d43ba8e594f91497fea981579656331276',
  '2020.01.22'],
 [5,
  '테넷',
  2000000,
  'https://search1.kakaocdn.net/thumb/R232x328.q85/?fname=https%3A%2F%2Ft1.daumcdn.net%2Fmovie%2F45ee167dd5a1fbbadc03af21dcba065324132c27',
  '2020.08.26'],
 [6,
  '백두산',
  8250000,
  'https://search1.kakaocdn.net/thumb/R232x328.

In [87]:
# csv 파일 저장
with open("movies.csv", "w", encoding="UTF-8", newline='') as f:
  writer = csv.writer(f)

  writer.writerows(mdata)